In [1]:
import yaml
import torch
import torch.backends.cudnn as cudnn
from argparse import Namespace
import pytorch_lightning as pl
import pandas as pd

from components.datamodule import DataModule
from components.pl_wrapper import Model
from vitstr import ModelConfig

In [4]:
"""load configuration"""
opt = Namespace(**yaml.safe_load(open("components/config.yaml", "r")))

""" Seed and GPU setting """
pl.seed_everything(opt.manualSeed)

cudnn.benchmark = True
cudnn.deterministic = True
opt.num_gpu = torch.cuda.device_count()

Global seed set to 230321


In [6]:
cfg = ModelConfig()
# Merge config with opt
for k, v in vars(cfg).items():
    if k not in vars(opt):
        setattr(opt, k, v)
if opt.kor:
    kor_character = "0123456789가강거경계고관광구금기김나남너노누다대더도동두등라러로루리마머명모무문미바배버보부북사산서소수시아악안양어연영오용우울원육이인자작저전조주중지차천초추충카타파평포하허호홀히"
    opt.character = kor_character
print(opt)
opt.saved_model = '../' + opt.saved_model

dm = DataModule(opt)
model = Model.load_from_checkpoint(opt.saved_model, opt=opt).eval()
model.freeze()

trainer = pl.Trainer(
    accelerator="auto",
    devices=opt.num_gpu,
    precision=16,
)

predict_result = trainer.predict(model, dm)

Namespace(adam=False, batch_max_length=10, batch_size=128, beta1=0.9, character='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ', data_root='/home/fourind/projects/datas/indonesia-datas/', eps=1e-08, exp_name=None, grad_clip=5, imgH=224, imgW=224, img_size=(224, 224), kor=False, lr=1, manualSeed=230321, model_name='vitstr_base_patch16_224', num_epoch=1000, num_fiducial=20, num_gpu=1, pad=False, pretrained=False, rgb=False, rho=0.95, saved_model='saved_models/ViTSTR-Seed230321/epoch=24-val-acc=98.624.ckpt', scheduler=True, swa_epoch_start=5, swa_lrs=0.01, workers=24)


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/fourind/projects/ViTSTR2/scripts/lightning_logs


Predict dataset size: 30772


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [8]:
predict_result = [p for predict in predict_result for p in predict]

In [9]:
len(predict_result)

30772

In [20]:
predict_df = pd.DataFrame(
    [
        (
            img_name.split(".jpg")[0].split("-")[0],
            pred.upper(),
            conf.item(),
        )
        for img_name, pred, conf in predict_result
    ],
    columns=['label', 'pred', 'conf'],
)
predict_df["correct"] = predict_df.apply(lambda x: x.label == x.pred, axis=1)

In [21]:
predict_df

,label,pred,conf,correct
0,YX4745XBL,YX4745XBL,0.478169,True
1,NJ9295OZJ,NJ9295OZJ,0.869117,True
2,XR6144BFO,XR6144BFO,0.479908,True
3,FQ9569VDS,FQ9569VDS,0.312318,True
4,JI7087ULH,JI7087ULH,0.849357,True
...,...,...,...,...
30767,F8602GL,F8002GL,0.662839,False
30768,QC7872UAT,QC7872UAT,0.407175,True
30769,LJ4980YXP,LJ4980YXP,0.329106,True
30770,SU0771BSL,SU0771BSL,0.506285,True


In [22]:
predict_df.conf.describe()

count    30772.000000
mean         0.527887
std          0.265479
min          0.013008
25%          0.292346
50%          0.489970
75%          0.763499
max          0.999900
Name: conf, dtype: float64

In [23]:
failures = predict_df.loc[predict_df["correct"] == False]

In [24]:
failures

,label,pred,conf,correct
204,FH9258KTI,FH9258KTT,0.198354,False
794,B1559FZI,B1959FZI,0.245729,False
878,F1686BF,F1186BF,0.600045,False
911,F1938HY,F1938NY,0.157552,False
934,B9622BXS,B9622AS,0.155904,False
...,...,...,...,...
30661,F1097RN,F1397RN,0.406754,False
30672,B2580ZAT,B2580TZA,0.426878,False
30720,D8123EE,D8137EE,0.435694,False
30762,F8444MS,F8444HE,0.303553,False


In [34]:
correct = predict_df.correct.value_counts()

In [36]:
acc = correct[True] / (correct[True] + correct[False])
acc

0.9859612634862862